In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing libraries

In [4]:
import requests
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
import json
from collections import Counter
from math import log
from math import sqrt
import heapq

import nltk
from nltk.tokenize import word_tokenize

import sys
# also look in this specific directory when searching for modules
sys.path.append(HW_path_folder)

import crawler # crawler.py
import parser # parser.py
import textprocessor # textprocessor.py
import engine # engine.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Defining some paths

In [5]:
HW_path_folder = "/content/drive/MyDrive/lavoro/data science/Sapienza/Algorithmic Methods of Data Mining/Homework 03"

# Set the path of the restaurants_urls.txt
TXT_restaurants_urls_file_path = os.path.join(HW_path_folder, "restaurants_urls.txt")

# Set the path of the folder for the HTML files of the restaurants
HTML_restaurants_files_path = os.path.join(HW_path_folder, "HTML_restaurants_files")

# TSV file with the entire dataset
TSV_restaurants = os.path.join(HW_path_folder, "restaurants.tsv")

# defining the path of the folder that will contain the restaurant_i.tsv files
TSV_files_restaurant_i_path = os.path.join(HW_path_folder, "TSV_files_restaurant_i")

# vocabulary.csv
vocabulary_filepath = os.path.join(HW_path_folder, "vocabulary.csv")

# inverted_index.json
inverted_index_filepath = os.path.join(HW_path_folder, "inverted_index.json")

# inverted_index_tfidf.json
inverted_index_tfidf_filepath = os.path.join(HW_path_folder, "inverted_index_tfidf.json")

# 1. Data Collection

## 1.1. Get the list of Michelin restaurants

Web scraping the [Michelin Restaurants in Italy](https://guide.michelin.com/en/it/restaurants) site, collecting the URL associated with each restaurant.

The output is saved in **restaurants_urls.txt**, where each line contains two *comma-separated* values:

*   number of page; for example:
  * number **2** for restaurants in page https://guide.michelin.com/en/it/restaurants/page/2
  * number **3** for restaurants in page https://guide.michelin.com/en/it/restaurants/page/3
  * ...
*   restaurant URL (e.g. https://guide.michelin.com/en/campania/gragnano/restaurant/o-me-o-il-mare)

We use a **get_restaurant_urls()** function defined in the module **crawler.py**





In [11]:
base_url = "https://guide.michelin.com/en/it/restaurants"
all_restaurant_urls = []
page_num = 1

while True:
    # Construct the URL for the current page
    if page_num == 1:
        url = base_url  # First page has a different URL structure
    else:
        url = f"{base_url}/page/{page_num}"

    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract restaurant URLs from the current page using the crawler.py
        restaurant_urls = crawler.get_restaurant_urls(soup)

        # Store page number and complete URL together
        for restaurant_url in restaurant_urls:
            complete_url = 'https://guide.michelin.com' + restaurant_url
            all_restaurant_urls.append((page_num, complete_url))

        print(f"Scraped page {page_num}")
        page_num += 1  # Move to the next page

        # Check if we've reached the end of the pages (no more restaurants found)
        if not restaurant_urls:
            break

    except requests.exceptions.RequestException as e:
        print(f"Error scraping page {page_num}: {e}")
        break  # Exit the loop in case of an actual error during page fetching

# Write to file
with open('restaurants_urls.txt', 'w') as f:
    for page_num, url in all_restaurant_urls:
        f.write(f"{page_num},{url}\n")  # Write page number and complete URL separated by a comma

print("URLs saved to restaurants_urls.txt")

Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Scraped page 21
Scraped page 22
Scraped page 23
Scraped page 24
Scraped page 25
Scraped page 26
Scraped page 27
Scraped page 28
Scraped page 29
Scraped page 30
Scraped page 31
Scraped page 32
Scraped page 33
Scraped page 34
Scraped page 35
Scraped page 36
Scraped page 37
Scraped page 38
Scraped page 39
Scraped page 40
Scraped page 41
Scraped page 42
Scraped page 43
Scraped page 44
Scraped page 45
Scraped page 46
Scraped page 47
Scraped page 48
Scraped page 49
Scraped page 50
Scraped page 51
Scraped page 52
Scraped page 53
Scraped page 54
Scraped page 55
Scraped page 56
Scraped page 57
Scraped page 58
Scraped page 59
Scraped page 60
Scraped page 61
Scraped page 62
Scraped page 63
S

## 1.2. Crawl Michelin restaurant pages

Now that we have the **restaurants_urls.txt**, we can:


* Download the HTML corresponding to each of the collected URLs
* Save them in a file immediately (this way, if the program stops for any reason, we will not lose the data collected up to the stopping point)
* Organize the donloaded HTML files into folders (each folder will contain the HTML of the restaurants from page 1, page 2, ... of the Michelin restaurant list)

We use a **download_restaurants_html_pages()** function defined in the module **crawler.py**

In [ ]:
# Call the function to download the pages, using crawler.py
crawler.download_restaurants_html_pages(TXT_restaurants_urls_file_path, HTML_restaurants_files_path)
# (runtime is about 8 mins)

Downloaded https://guide.michelin.com/en/campania/gragnano/restaurant/o-me-o-il-mare to /content/drive/MyDrive/lavoro/data science/Sapienza/Algorithmic Methods of Data Mining/Homework 03/HTML_restaurants_files/1/o-me-o-il-mare.html
Downloaded https://guide.michelin.com/en/abruzzo/popoli_1845563/restaurant/donevandro to /content/drive/MyDrive/lavoro/data science/Sapienza/Algorithmic Methods of Data Mining/Homework 03/HTML_restaurants_files/1/donevandro.html
Downloaded https://guide.michelin.com/en/piemonte/alba/restaurant/ape-vino-e-cucina to /content/drive/MyDrive/lavoro/data science/Sapienza/Algorithmic Methods of Data Mining/Homework 03/HTML_restaurants_files/1/ape-vino-e-cucina.html
Downloaded https://guide.michelin.com/en/campania/sorrento/restaurant/da-bob-cook-fish to /content/drive/MyDrive/lavoro/data science/Sapienza/Algorithmic Methods of Data Mining/Homework 03/HTML_restaurants_files/1/da-bob-cook-fish.html
Downloaded https://guide.michelin.com/en/basilicata/matera/restaurant

## 1.3 Parse downloaded pages

Now that we have all the HTML documents about the restaurants, we can extract these informations:

1. **Restaurant Name** (to save as restaurantName): string;
2. **Address** (to save as address): string;
3. **City** (to save as city): string;
4. **Postal Code** (to save as postalCode): string;
5. **Country** (to save as country): string;
6. **Price Range** (to save as priceRange): string;
7. **Cuisine Type** (to save as cuisineType): string;
8. **Description** (to save as description): string;
9. **Facilities and Services** (to save as facilitiesServices): list of strings;
10. **Accepted Credit Cards** (to save as creditCards): list of strings;
11. **Phone Number** (to save as phoneNumber): string;
12. **URL to the Restaurant Page** (to save as website): string.

Let's save the dataset in a **restaurants.tsv** file, to keep it in an easily accessible format.

In [ ]:
all_restaurants_data = [] # List to store data for all restaurants
restaurant_index = 0 # List to store data for all restaurants

total_number_of_folders = 100 # total number of folders containing the HTML files (1 2 3 4 ... 100)

#for subdir in range(1, 2):
for subdir in range(1, total_number_of_folders + 1):
  subdir_path = os.path.join(HTML_restaurants_files_path, str(subdir))
  for filename in os.listdir(subdir_path):
    if filename.endswith(".html"):

      # print the number of the restaurant during the runtime
      print("restaurant_index:", restaurant_index)

      html_file = os.path.join(subdir_path, filename)
      # Extract restaurant info using parser.py module
      restaurant_info = parser.extract_restaurant_info(html_file)

      # print the restaurant infos during the runtime
      # print("restaurant_info:", restaurant_info)

      # Add restaurant data to the list
      all_restaurants_data.append(restaurant_info)

      restaurant_index += 1

# Create a Pandas DataFrame
df = pd.DataFrame(all_restaurants_data, columns=[
    'restaurantName', 'address', 'city', 'postalCode', 'country',
    'priceRange', 'cuisineType', 'description', 'facilitiesServices',
    'creditCards', 'phoneNumber', 'website'
])

# DataFrame >>> restaurants.tsv file
df.to_csv(TSV_restaurants, sep='\t')  # Use '\t' for tab separation

restaurant_index: 0
restaurant_index: 1
restaurant_index: 2
restaurant_index: 3
restaurant_index: 4
restaurant_index: 5
restaurant_index: 6
restaurant_index: 7
restaurant_index: 8
restaurant_index: 9
restaurant_index: 10
restaurant_index: 11
restaurant_index: 12
restaurant_index: 13
restaurant_index: 14
restaurant_index: 15
restaurant_index: 16
restaurant_index: 17
restaurant_index: 18
restaurant_index: 19
restaurant_index: 20
restaurant_index: 21
restaurant_index: 22
restaurant_index: 23
restaurant_index: 24
restaurant_index: 25
restaurant_index: 26
restaurant_index: 27
restaurant_index: 28
restaurant_index: 29
restaurant_index: 30
restaurant_index: 31
restaurant_index: 32
restaurant_index: 33
restaurant_index: 34
restaurant_index: 35
restaurant_index: 36
restaurant_index: 37
restaurant_index: 38
restaurant_index: 39
restaurant_index: 40
restaurant_index: 41
restaurant_index: 42
restaurant_index: 43
restaurant_index: 44
restaurant_index: 45
restaurant_index: 46
restaurant_index: 47
re

For each restaurant, we have to create a **restaurant_i.tsv** file of this structure:

`restaurantName \t address \t  ... \t url`

Starting from the Pandas DataFrame created, we save the **restaurant_i.tsv** files in the folder **/TSV_files_restaurant_i**

In [ ]:
# iterates over the DataFrame rows
for i in range(len(df)):
  # defining the file path for each TSV file (e.g. /TSV_files_restaurant_i/restaurant_0.tsv
  #                                                /TSV_files_restaurant_i/restaurant_1.tsv
  #                                                /TSV_files_restaurant_i/restaurant_2.tsv
  #                                                                                    ...)
  restaurant_i_file_path = os.path.join(TSV_files_restaurant_i_path, f"restaurant_{i}.tsv")

  # extracts the i-th row and saves it to a .tsv file
  df.iloc[[i]].to_csv(restaurant_i_file_path, sep='\t', index=False)

# 2. Search Engine

This search engine allows us to retrieve restaurants based on a user query. We’ll build two types of search engines:

1. **Conjunctive Search Engine**: Returns restaurants where all query terms appear in the description.
2. **Ranked Search Engine**: Returns the top-k restaurants sorted by similarity to the query, using TF-IDF and Cosine Similarity.

## 2.0 Preprocessing

Before building the search engine, we must clean and prepare the text in each restaurant’s description.

We will use the [nltk library](https://www.nltk.org/), adding a new column to the DataFrame with the *description* processed-text (**'description_cleaned'**).

In [12]:
# creating the Data Frame importing "restaurants.tsv" file
df = pd.read_csv(TSV_restaurants, sep='\t')

Using the *cleaner* function that we have defined in the **textprocessor.py** module

In [13]:
# Initialize an empty list to store cleaned descriptions
descriptions_cleaned = []

# Iterate over each description in the 'description' column of the DataFrame
for description in df['description']:

  cleaned_desc = textprocessor.cleaner(description)

  # Add the cleaned description to the list
  descriptions_cleaned.append(cleaned_desc)

# Add the cleaned descriptions as a new column in the DataFrame
df['description_cleaned'] = descriptions_cleaned

Comparison between the original text and the *processed* one

In [14]:
# Display "description" and "description_cleaned", just to few raws
df[['restaurantName', 'description', 'description_cleaned']].head(3)

,restaurantName,description,description_cleaned
0,O Me O Il Mare,"Known around the world as the town of pasta, G...",known around world town pasta gragnano still h...
1,Donevandro,"Up until a few years ago, the owner-chef at th...",year ago owner chef restaur work painter fact ...
2,Ape Vino e Cucina,This attractive restaurant in the heart of Alb...,attract restaur heart alba gastronom capit lan...


## 2.1 Conjunctive Query

This first version of the search engine narrows the search to the description field of each restaurant. Only restaurants whose **descriptions** contain all the query words will be returned.

### 2.1.1 Create Your Index!

1. **Vocabulary File**: Create a file called vocabulary.csv that maps each word to a unique integer (*term_id*).

2. **Inverted Index**: Build a dictionary mapping each *term_id* to a list of document IDs where that term appears.

Creating **vocabulary.csv**

In [15]:
# an empty set to store the vocabulary
vocabulary = set()

# for loop over the cleaned descriptions in the DataFrame
for description in df['description_cleaned']:
    # Split each description into individual words
    words = description.split()
    # Add the words to the vocabulary set; duplicates are automatically ignored because we are using a set()
    vocabulary.update(words)

# Create a dictionary mapping terms to term IDs using enumerate to assign IDs
vocabulary_dictionary = {term: term_id for term_id, term in enumerate(vocabulary)}

# Save the vocabulary to 'vocabulary.csv'
with open(vocabulary_filepath, 'w', encoding='utf-8') as f:
    for term, term_id in vocabulary_dictionary.items():
        f.write(f"{term},{term_id}\n")

Creating the **Inverted Index**

In [16]:
def create_inverted_index(df, vocabulary_dict):

    # Creates an inverted index from the cleaned descriptions and vocabulary

    inverted_index = {} # empty dictionary
    for doc_id, description in enumerate(df['description_cleaned']):
        words = description.split()
        for word in words:
            term_id = vocabulary_dict.get(word)
            if term_id is not None:
                if term_id not in inverted_index:
                    inverted_index[term_id] = []
                inverted_index[term_id].append(doc_id)

    return inverted_index

# Create the inverted index
inverted_index = create_inverted_index(df, vocabulary_dictionary)

# Save the inverted index to a JSON file
import json
with open(inverted_index_filepath, 'w', encoding='utf-8') as jsonfile:
    json.dump(inverted_index, jsonfile)

### 2.1.2 Execute the Query

When the user inputs a query, for example, "modern seasonal cuisine", the search engine will:

1. Process the query terms
2. Find and return a list of restaurants containing **all** query words in their description

The output should include:

* restaurantName
* address
* description
* website



We define the query "modern seasonal cuisine" and we process it

In [17]:
query = "modern seasonal cuisine"
print("Query:", query)
processed_query = textprocessor.cleaner(query).split()  # Use cleaner from textprocessor
print("Processed query:", processed_query)

Query: modern seasonal cuisine
Processed query: ['modern', 'season', 'cuisin']


In [20]:
# importing vocabulary.csv as DataFrame
voc_df = pd.read_csv(vocabulary_filepath, names=['term', 'term_id'])
# importing Inverted Index
with open(inverted_index_filepath, 'r', encoding='utf-8') as jsonfile:
  inverted_index = json.load(jsonfile)
# importing restaurants.tsv
df = pd.read_csv(TSV_restaurants, sep='\t')

result_df = engine.conjunctive_search(processed_query, voc_df, inverted_index, df)
result_df[['restaurantName', 'address', 'description', 'website']]

Common numbers: [17, 30, 84, 301, 325, 329, 416, 439, 487, 636, 637, 680, 807, 890, 913, 934, 996, 1065, 1074, 1144, 1203, 1281, 1313, 1323, 1439, 1546, 1555, 1570, 1619, 1678, 1695, 1725, 1789, 1813, 1814, 1816, 1840, 1884, 1915, 1943]


,restaurantName,address,description,website
17,20Tre,via David Chiossone 20 r,"Run by three partners, this contemporary-style...",https://www.ristorante20tregenova.it/
30,Casin del Gamba,via Roccolo Pizzati 1,The journey to get here – a winding road throu...,https://www.casindelgamba.it/
84,San Giorgio,viale Brigate Bisagno 69r,Situated in the city albeit not right in the c...,https://www.ristorantesangiorgiogenova.it/
301,Ca' Del Moro,località Erbin 31,Situated within the La Collina dei Ciliegi win...,https://www.cadelmoro.wine/it
325,Contrasto,via Roma 55,"Having returned to his native village, owner-c...",https://contrastoristorante.it
329,Saur,via Filippo Turati 8,"In a tiny rural village, this contemporary, al...",https://ristorantesaur.it
416,San Michele,via Castello di Fagagna 33,Situated next to the ruins of the old castle a...,http://sanmichele.restaurant
439,Chichibio,via Guglielmo Marconi 1,"Despite its lack of awards, this restaurant st...",NaN
487,Winter Garden Florence,piazza Ognissanti 1,Horse-drawn carriages once entered the old cou...,https://www.wintergardenflorence.com/it/
636,La Valle,"via Umberto I 25, località Valle Sauglio",A well - run restaurant in a quiet area just o...,https://www.ristorantelavalle.it/


## 2.2 Ranked Search Engine with TF-IDF and Cosine Similarity

For the second search engine, given a query, retrieve the top-k restaurants ranked by relevance to the query.

### 2.2.1 Inverted Index with TF-IDF Scores

1. **tfIdf Scores**: Calculate TF-IDF scores for each term in each restaurant’s description.

2. **Updated Inverted Index**: Build a new inverted index where each entry is a term, and the value is a list of tuples containing document IDs and TF-IDF scores.

Re-create the *description_cleaned* in the DataFrame (it will be useful for the next code)

In [23]:
# Initialize an empty list to store cleaned descriptions
descriptions_cleaned = []
# Iterate over each description in the 'description' column of the DataFrame
for description in df['description']:
  cleaned_desc = textprocessor.cleaner(description)
  # Add the cleaned description to the list
  descriptions_cleaned.append(cleaned_desc)
# Add the cleaned descriptions as a new column in the DataFrame
df['description_cleaned'] = descriptions_cleaned

In [24]:
def build_inverted_index_tfidf(df):

    # 1. Calculate TF
    term_frequencies = []
    for index, row in df.iterrows():
        words = row['description_cleaned'].split()
        tf = Counter(words)
        for term in tf:
            tf[term] /= len(words)
        term_frequencies.append(tf)

    # 2. Calculate IDF
    idf = {}
    num_docs = len(df)
    for index, row in df.iterrows():
        words = set(row['description_cleaned'].split())
        for word in words:
            idf[word] = idf.get(word, 0) + 1
    for word in idf:
        idf[word] = log(num_docs / idf[word])

    # 3. Calculate TF-IDF and build the inverted index
    inverted_index_tfidf = {}
    for i, row in df.iterrows():
        words = row['description_cleaned'].split()
        for word in words:
            tfidf = term_frequencies[i].get(word, 0) * idf.get(word, 0)
            if word not in inverted_index_tfidf:
                inverted_index_tfidf[word] = []
            inverted_index_tfidf[word].append((i, tfidf))

    return inverted_index_tfidf

# Build the inverted index with TF-IDF scores
inverted_index_tfidf = build_inverted_index_tfidf(df)

# Save the inverted index with TF-IDF scores to a json file
with open(inverted_index_tfidf_filepath, 'w', encoding='utf-8') as f:
    json.dump(inverted_index_tfidf, f)

### 2.2.2 Execute the Ranked Query

For the ranked search engine:

1. Process the query terms.
2. Use Cosine Similarity to rank matching restaurants based on the TF-IDF vectors of the query and each document.
3. Return the top-k results or all matching restaurants if fewer than k have non-zero similarity.

Each result should include:

* restaurantName
* address
* description
* website
* Similarity score (between 0 and 1)

We re-define the query ("modern seasonal cuisine" for example) and we process it

In [25]:
query = "modern seasonal cuisine"
print("Query:", query)
processed_query = textprocessor.cleaner(query).split()  # Use cleaner from textprocessor
print("Processed query:", processed_query)

Query: modern seasonal cuisine
Processed query: ['modern', 'season', 'cuisin']


In [26]:
# Load the inverted index with TF-IDF scores
with open(inverted_index_tfidf_filepath, 'r', encoding='utf-8') as f:
    inverted_index_tfidf = json.load(f)

results = engine.ranked_search(processed_query, inverted_index_tfidf, df)

# Save the results in a new DataFrame
result_df = pd.DataFrame(results)

# Print the specified columns from the DataFrame
print(result_df[['restaurantName', 'address', 'description', 'website', 'similarity']].sort_values(by='similarity', ascending=False).to_markdown(index=False, numalign="left", stralign="left"))

| restaurantName      | address                     | description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | website                                           | similarity   |
|:--------------------|:----------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 3. Define a New Score!

Now, we will define a custom ranking metric to prioritize restaurants based on user queries.

***Steps:***

1. **User Query**: The user provides a text query. We’ll retrieve relevant documents using the search engine built in Step 2.1.
2. **New Ranking Metric**: After retrieving relevant documents, we’ll rank them using a new custom score. Instead of limiting the scoring to only the 'description' field, we can include other attributes like *priceRange*, *facilitiesServices*, and *cuisineType*.
3. You will use a **heap data structure** (e.g., Python’s *heapq* library) to maintain the top-k restaurants.

**New Scoring Function:**

Define a scoring function that takes into account various attributes:

* **Description Match**: Give weight based on the query similarity to the description (using TF-IDF scores).
* **Cuisine Match**: Increase the score for matching cuisine types.
* **Facilities and Services**: Give more points for matching facilities/services (e.g. “Terrace“, "Air conditioning”).
* **Price Range**: Higher scores could be given to more affordable options based on the user’s choice.

**Output:**

The output should include:

* restaurantName
* address
* description
* website
* The new similarity score based on the custom metric.

Are the results you obtain better than with the previous scoring function? **Explain and compare results.**

We define a test-query: "italian contemporary cuisine wheelchair"

In [27]:
query = "italian contemporary cuisine wheelchair"
print("Query:", query)
processed_query = textprocessor.cleaner(query).split()  # Use cleaner from textprocessor
print("Processed query:", processed_query)

Query: italian contemporary cuisine wheelchair
Processed query: ['italian', 'contemporari', 'cuisin', 'wheelchair']


We use the *calculate_similarity* and *calculate_score* functions in **engine.py** module to calculate the new score.

The results of this new approach are compared with the previous ones, produced by the *conjunctive search*.

In the new score we apply the similarity score to each restaurants that we get after the use of *conjunctive_search*, then we calculate the new score applying some overall arbitrary weights to the values in 'description_match' (0.2), 'cuisine_match' (0.3), facilities_and_services' (0.3) and 'price_range' (0.2) favoring:
* the occurrences of query words in 'cuisineType' and 'facilitiesServices'
* the cheapest restaurants



In [28]:
# importing vocabulary.csv as DataFrame
voc_df = pd.read_csv(vocabulary_filepath, names=['term', 'term_id'])
# importing Inverted Index
with open(inverted_index_filepath, 'r', encoding='utf-8') as jsonfile:
  inverted_index = json.load(jsonfile)
# importing restaurants.tsv
df = pd.read_csv(TSV_restaurants, sep='\t')

### Adding a 'description_cleaned' column to the DataFrame, as done previous ###
# Initialize an empty list to store cleaned descriptions
descriptions_cleaned = []
# Iterate over each description in the 'description' column of the DataFrame
for description in df['description']:
  cleaned_desc = textprocessor.cleaner(description)
  # Add the cleaned description to the list
  descriptions_cleaned.append(cleaned_desc)
# Add the cleaned descriptions as a new column in the DataFrame
df['description_cleaned'] = descriptions_cleaned

# Perform conjunctive search using the engine.py
result_df = engine.conjunctive_search(processed_query, voc_df, inverted_index, df)

# defining a new DataFrame that we will use for the new score
result_df_newscore = result_df.copy()

# Calculate the new score for each restaurant and use a heap to find the top-k
k = 10  # Number of top restaurants to return
heap = [] # Initialize an empty heap
for index, row in result_df_newscore.iterrows(): # Iterate over each row in the DataFrame
    restaurant = row.to_dict() # Convert the row to a dictionary

    # Process the 'cuisineType' field
    restaurant['cuisineType'] = textprocessor.cleaner(restaurant['cuisineType'])
    # Process the 'facilitiesServices' field
    restaurant['facilitiesServices'] = textprocessor.cleaner(restaurant['facilitiesServices'])

    # Calculate the similarity score
    similarity = engine.calculate_similarity(restaurant['description_cleaned'], processed_query, inverted_index_tfidf)

    # Calculate the overall score
    score = engine.calculate_score(restaurant, processed_query, similarity)
    # Push the score and restaurant onto the heap (use -score for max-heap)
    heapq.heappush(heap, (-score, restaurant))

top_k_restaurants = [] # empty list for top_k restaurants
for _ in range(k):

    score, restaurant = heapq.heappop(heap)  # Get both score and restaurant
    restaurant['score'] = -score  # Add the score to the restaurant dictionary with correct sign
    top_k_restaurants.append(restaurant)

# Display the results

### OUTPUT 1 (just conjunctive search) ###
columns_to_display = ['restaurantName', 'address', 'cuisineType', 'priceRange', 'facilitiesServices', 'description']
result_df = result_df[columns_to_display]
print("\n")
print("Conjunctive Search Results:\n")
print(result_df.to_markdown(index=False, numalign="left", stralign="left"))
print("\n\n")

### OUTPUT 2 (with new score) ###
result_df_newscore = pd.DataFrame(top_k_restaurants)
columns_to_display = ['restaurantName', 'address', 'cuisineType', 'priceRange', 'facilitiesServices', 'description', 'score']
result_df_newscore = result_df_newscore[columns_to_display]
print("Ranked Search Results (with new score):\n")
print(result_df_newscore.to_markdown(index=False, numalign="left", stralign="left"))

Common numbers: [88, 104, 172, 233, 257, 353, 401, 449, 538, 579, 707, 965, 991, 1001, 1121, 1132, 1223, 1256, 1277, 1466, 1517, 1667, 1836, 1929]


Conjunctive Search Results:

| restaurantName                        | address                             | cuisineType                            | priceRange   | facilitiesServices                                                                                                            | description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

# Algorithmic Question (AQ)

A robot is in a warehouse represented by a coordinate grid and needs to collect $n$ packages. It starts at $(0,0)$, and the *i-th* package is at $(x_i,y_i)$. No two packages are at the same coordinates, and $(0,0)$ is empty. The robot can only move up ('**U**') or right ('**R**'), either from $(x,y)$ to $(x+1,y)$ or $(x,y+1)$. The goal is to collect all $n$ packages with the fewest moves, choosing the lexicographically smallest path if multiple shortest paths exist.

**Input**

* The first line contains $t(1≤t≤10)$ — the number of test cases.
* Each test case starts with $n(1≤n≤100)$, the number of packages.
* The next $n$ lines contain the coordinates $x_i,y_i(0 ≤ x_i, y_i ≤ 100)$ for each package.

**Output**

* For each test case, print "YES" and the lexicographically smallest path, or "NO" if it’s impossible to collect all packages.


---
1. Write the **pseudocode** for an algorithm that solves this problem

---

Initialize the robot's starting position at (0, 0):
* $currentX = 0$
* $currentY = 0$

Initialize an empty string to store the path:
* $path = ""$

Sort the packages by x-coordinate, then y-coordinate.

For each package $(x,y)$:
* check if it is unreachable
  * if $x < currentX$ OR $y < currentY$ **return** NO
* calculate the difference between $X$ and $currentX$
  * the result represents the number of $R$ to append to the $path$
* calculate the difference between $Y$ and $currentY$
  * the result represents the number of $U$ to append to the $path$
* update the robot's position
  * $currentX = X$
  * $currentY = Y$

**Return** YES and the $path$


---
2. Prove that your algorithm is correct
---

In this case the *lexicographically smallest path* refers to the shortest path for collecting all the packages, which also has the smallest value when paths are sorted alphabetically (where 'R' comes before 'U').

E.G. $ RRRUU $ is *lexicographically* smaller than $ RRUUU $ (the first difference between the two paths is at the third letter, with $R < U$)



In [29]:
def robot_path(packages):

    packages.sort()  # Sort the packages by x-coordinate, then y-coordinate.
    path = ""  # Initialize an empty string to store the path.
    current_x, current_y = 0, 0  # Start the robot at coordinates (0, 0).

    for x, y in packages:  # Iterate through each package in the sorted list.
        if x < current_x or y < current_y:  # Check if the package is unreachable.
            return "NO"  # If unreachable, return "NO".

        path += "R" * (x - current_x)  # Move right to the package's x-coordinate.
        path += "U" * (y - current_y)  # Move up to the package's y-coordinate.
        current_x, current_y = x, y  # Update the robot's current position.

    return "YES " + path  # If all packages are reachable, return "YES" and the path.


# Get the number of test cases from the user.
t = int(input("Enter the number of test cases: "))
for test in range(t):  # Loop through each test case.
    n = int(input("Enter the number of packages: "))  # Get the number of packages.
    packages = []  # Initialize an empty list to store the package coordinates.
    for i in range(n):  # Loop to get the coordinates of each package.
        x, y = map(int, input(f"Enter coordinates of {i+1}-th package: ").split())  # Get x and y coordinates.
        packages.append((x, y))  # Add the coordinates as a tuple to the packages list.

    print(robot_path(packages))  # Call the function to calculate and print the path.

Enter the number of test cases: 3
Enter the number of packages: 5
Enter coordinates of 1-th package: 1 3
Enter coordinates of 2-th package: 1 2
Enter coordinates of 3-th package: 3 3
Enter coordinates of 4-th package: 5 5
Enter coordinates of 5-th package: 4 3
YES RUUURRRRUU
Enter the number of packages: 2
Enter coordinates of 1-th package: 1 0
Enter coordinates of 2-th package: 0 1
NO
Enter the number of packages: 1 
Enter coordinates of 1-th package: 4 3
YES RRRRUUU


---
3. Compute the time complexity of your algorithm in Big O notation. Break down the steps involved in the algorithm, and explain which parts contribute most to the overall time complexity
---

* *packages.sort()* uses Python's built-in sorting algorithm (Timsort) with **O (n log n)** time complexity
* *for x, y in packages* loop iterates through each package once, performing a constant amount of work for each package, so time complexity is **O (n)**
* *path += "R" (x - current_x)* and *path += "U" (y - current_y)*, depending by the dimension of the grid (where the package is positioned?), contribute with **O (1)** time complexity
* other operations with **O (1)** time complexity (initializing, checking, updating variables)

The overall time complexity of the algorithm is **O (n log n)**


---
4. Ask an LLM tool (such as ChatGPT, Claude AI, Gemini, Perplexity, etc.) to evaluate the time complexity of your code using Big O notation. Is the assessment accurate? If it differs from your previous analysis, which would be correct? Please explain your reasoning
---

Below the response of **Claude Haiku**, that seems correct and accurate.

*\[PROMPT: Evaluate the time complexity of this code using Big O notation\]*

```
I'll analyze the time complexity of the robot_path function using Big O notation:
Let's break down the time complexity of each step:

packages.sort():

Python's built-in sort() method uses Timsort, which has a time complexity of O(n log n), where n is the number of packages.

Iterating through packages:

The for loop runs once for each package, so it's O(n).
Inside the loop, the operations are constant time:

Comparisons: O(1)
String concatenation for "R" and "U" moves: O(1)
Updating current coordinates: O(1)

Overall time complexity:

Sorting: O(n log n)
Iteration: O(n)
Combining these, the dominant term is O(n log n)

Therefore, the overall time complexity of the robot_path function is O(n log n), primarily due to the sorting operation.

```



---
5. Assume now that the robot can also move towards the left or downwards, and consider the greedy approach: from the current location go to the closest package. Notice that now we can always collect all packages. Prove that the greedy algorithm is optimal (i.e., it minimizes the total distance traveled), or provide a counterexample showing that it is not
---


The greedy approach of going to the closest package does not guarantee an optimal solution in this case.

The greedy approach gets "stuck" by prioritizing immediate closeness at each step, at the expense of the best overall solution.

An example below:

```
3 packages: in (0,4), (1,2) and (4,0)

.(0,4)

.

.

.______.(1,2)

.(0,0) .______.______.______.(4,0)
```


|**Greedy approach**|**A more efficient approach**|
|:-----------------:|:---------------------------:|
|1. move to (1,2): 3 steps|1. move to (0,4): 4 steps|
|2. move to (0,4): 3 steps|2. move to (1,2): 3 steps|
|3. move to (4,0): 8 steps|3. move to (4,0): 5 steps|
|**TOT STEPS: 14**|**TOT STEPS: 12**|